In [1]:
!pip install langgraph

In [2]:
import boto3
from botocore.config import Config

region_name = "us-east-1"
#llm_model = "anthropic.claude-3-sonnet-20240229-v1:0"
llm_model = "anthropic.claude-3-haiku-20240307-v1:0"

def converse_with_bedrock(sys_prompt, usr_prompt):
    temperature = 0.1
    top_p = 0.3
    top_k = 10
    inference_config = {"temperature": temperature, "topP": top_p}
    additional_model_fields = {"top_k": top_k}
    response = boto3_client.converse(
        modelId=llm_model, 
        messages=usr_prompt, 
        system=sys_prompt,
        inferenceConfig=inference_config,
        additionalModelRequestFields=additional_model_fields
    )
    return response['output']['message']['content'][0]['text']

def init_boto3_client(region: str):
    retry_config = Config(
        region_name=region,
        retries={"max_attempts": 10, "mode": "standard"}
    )
    return boto3.client("bedrock-runtime", region_name=region, config=retry_config)


def create_prompt(sys_template, user_template, **kwargs):
    sys_prompt = [{"text": sys_template.format(**kwargs)}]
    usr_prompt = [{"role": "user", "content": [{"text": user_template.format(**kwargs)}]}]
    return sys_prompt, usr_prompt

boto3_client = init_boto3_client(region_name)


In [3]:
from py2neo import Graph
import os

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "password"

graph = Graph()

In [4]:
from typing import TypedDict, List, Optional

class GraphState(TypedDict, total=False):
    question: str
    subgraph: str
    target_node: List[int]
    next_step: str
    node_pos: Optional[int]
    trace: List[str]

In [5]:
import json

csv_list_response_format = "Your response should be a list of comma separated values, eg: `foo, bar` or `foo,bar`"

def select_subgraph(state: GraphState) -> GraphState:
    question = state["question"]
    query = """
        MATCH (n:Title {level: "1"})
        RETURN n.value, id(n) as node_id
    """
    results = graph.run(query)
    subgraph_list = [(record["n.value"], record["node_id"]) for record in results]
    subgraph_list_with_number = [f"{i}. {subgraph[0]}" for i, subgraph in enumerate(subgraph_list)]

    sys_prompt_template = "당신은 AWS 매뉴얼 문서에 정통한 전문 엔지니어입니다.\n사용자의 질문에 가장 적절한 매뉴얼 문서 이름을 선택하는 것이 당신의 임무입니다. 관련된 문서가 없으면 빈 목록("")을 제공합니다."
    usr_prompt_template = "주어진 질문에 가장 관련성 높은 문서 이름 1개를 선택해주세요.\n#질문: {question}\n\n #문서 목록:\n {subgraph_list_with_number}\n\n #응답 형식: 선택된 문서의 인덱스 번호만 제공 (서두는 생략)"
    sys_prompt, usr_prompt = create_prompt(sys_prompt_template, usr_prompt_template, question=question, subgraph_list_with_number=subgraph_list_with_number)
    
    selected_id = converse_with_bedrock(sys_prompt, usr_prompt)
    try:
        if selected_id == "":
            return GraphState(next_step="generate_answer")

        else: 
            selected_subgraph_id = subgraph_list[int(selected_id)][1]
    except:
        return GraphState(next_step="generate_answer")
    return GraphState(target_node=[selected_subgraph_id], next_step="traverse_child", )

def traverse_child(state: GraphState) -> GraphState: 
    question = state["question"]
    node_id = state["target_node"][0]
    child_list = []

    query = """
        MATCH (n:Title)-[:HAS_CHILD]->(c)
        WHERE id(n) = $node_id
        RETURN c.value, id(c) as child_id
    """
    params = {"node_id": node_id}
    results = graph.run(query, params)
    child_list = [(record["c.value"], record["child_id"]) for record in results]
    if not child_list:
        return GraphState(next_step="get_contents")
    
    child_list_with_number = [f"{i}. {child}" for i, child in enumerate(child_list)]
    sys_prompt_template = "당신은 AWS 매뉴얼 문서에 정통한 전문 엔지니어입니다.\n사용자의 질문에 답변하기 위해 가장 적합한 하위 메뉴를 선택하는 것이 당신의 임무입니다."
    usr_prompt_template = "질문과 가장 관련성 높은 하위 메뉴를 3개 이하로 선택해주세요.\n선택 기준:\n1. 질문과 직접적으로 연관된 메뉴만 선택\n2. 가장 관련성 높은 순서대로 정렬\n3. 불필요하거나 관련 없는 메뉴는 제외\n#질문: {question}\n\n #메뉴 목록:\n {child_list_with_number}\n\n #응답 형식(선택된 메뉴의 인덱스 번호만 쉼표로 구분하여 나열): {csv_list_response_format}"
    sys_prompt, usr_prompt = create_prompt(sys_prompt_template, usr_prompt_template, question=question, child_list_with_number=child_list_with_number, csv_list_response_format=csv_list_response_format)
    selected_ids = converse_with_bedrock(sys_prompt, usr_prompt)
    try:
        if selected_ids == '""' or selected_ids.strip() == "":
            return GraphState(next_step="get_contents")
        else:
            selected_id_list = [int(id.strip()) for id in selected_ids.split(',') if id.strip().isdigit()]
            selected_childs = [child_list[id][1] for id in selected_id_list] if selected_id_list else []
            return GraphState(next_step="traverse_child", target_node=selected_childs)
    except:
        return GraphState(next_step="get_contents")

In [9]:
from langchain_aws import ChatBedrock
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

csv_list_response_format = "Your response should be a list of comma separated values, eg: `foo, bar` or `foo,bar`"

def select_subgraph_dev(question, graph):
    question = question
    query = """
        MATCH (n:Title {level: "1"})
        RETURN n.value, id(n) as node_id
    """
    results = graph.run(query)
    subgraph_list = [(record["n.value"], record["node_id"]) for record in results]
    subgraph_list_with_number = [f"{i}. {subgraph[0]}" for i, subgraph in enumerate(subgraph_list)]

    sys_prompt_template = "당신은 AWS 매뉴얼 문서에 정통한 전문 엔지니어입니다.\n사용자의 질문에 가장 적절한 매뉴얼 문서 이름을 선택하는 것이 당신의 임무입니다. 관련된 문서가 없으면 빈 목록("")을 제공합니다."
    usr_prompt_template = "주어진 질문에 가장 관련성 높은 문서 이름 1개를 선택해주세요.\n#질문: {question}\n\n #문서 목록:\n {subgraph_list_with_number}\n\n #응답 형식: 선택된 문서의 인덱스 번호만 제공 (서두는 생략)"
    sys_prompt, usr_prompt = create_prompt(sys_prompt_template, usr_prompt_template, question=question, subgraph_list_with_number=subgraph_list_with_number)
    
    selected_id = converse_with_bedrock(sys_prompt, usr_prompt)
    try:
        if selected_id == "":
            return [], "generate_answer"

        else: 
            selected_subgraph_id = subgraph_list[int(selected_id)][1]
            return [selected_subgraph_id], "traverse_child"
    except:
        return [], "generate_answer"

def traverse_child_dev(question, graph, target_node):
    question = question
    node_id = target_node[0]
    child_list = []
    query = """
        MATCH (n)
        WHERE id(n) = $node_id
        OPTIONAL MATCH (n)-[:HAS_CHILD]->(c)
        RETURN n.value as node_value, c.value, id(c) as child_id
    """
    params = {"node_id": node_id}
    results = graph.run(query, params)

    node_value = None
    child_list = []

    for record in results:
        if node_value is None:
            node_value = record["node_value"]
        if record["c.value"] is not None:
            child_list.append((record["c.value"], record["child_id"]))
    
    print(f"Node value: {node_value}")

    if not child_list:
        return node_id, [], "get_contents"

    child_list_with_number = [f"{i}. {child}" for i, child in enumerate(child_list)]
    sys_prompt_template = "당신은 AWS 매뉴얼 문서에 정통한 전문 엔지니어입니다.\n당신의 임무는 사용자의 질문에 답변하기 위해, 매뉴얼 문서에서 참고할 메뉴 항목을 선택하는 것입니다."
    usr_prompt_template = """
    질문과 연관된 하위 메뉴의 인덱스 번호를 최대 3개 선택한 후, 연관성이 높은 메뉴부터 내림차순으로 정렬하여 답변합니다. 
    질문에 포함된 키워드와 정확히 일치하는 메뉴 항목에 가장 높은 우선순위를 부여하세요. 
    더 구체적이고 특정한 주제를 다루는 메뉴 항목에 높은 우선순위를 부여하세요. 예를 들어, 일반적인 'Getting started' 가이드보다는 특정 기능이나 서비스에 대한 상세 설명이 있는 항목을 선호합니다. 
    질문의 전체적인 맥락을 고려하여 가장 적절한 메뉴 항목을 선택하세요. 단순히 키워드가 일치한다고 해서 반드시 가장 연관성이 높은 것은 아닙니다.
    
    #질문: {question}
     
    #메뉴 목록:
    {child_list_with_number}
     
    #응답 형식: {csv_list_response_format}. 
    (연관된 메뉴가 없으면 빈 목록("")으로 응답하세요)
    """
    sys_prompt, usr_prompt = create_prompt(sys_prompt_template, usr_prompt_template, question=question, child_list_with_number=child_list_with_number, csv_list_response_format=csv_list_response_format)
    selected_ids = converse_with_bedrock(sys_prompt, usr_prompt)
    try:
        if selected_ids == '""' or selected_ids.strip() == "":
            return node_id, [], "get_contents"
        else:
            selected_id_list = [int(id.strip()) for id in selected_ids.split(',') if id.strip().isdigit()]
            selected_childs = [child_list[id][1] for id in selected_id_list] if selected_id_list else []
            return node_id, selected_childs, "traverse_child"
    except:
        return node_id, [], "get_contents"

def get_contents_dev(question, cur_pos):
    question = question
    node_id = cur_pos

    count_query = """
        MATCH (n)-[:HAS_CONTENTS]->(c)
        WHERE id(n) = $node_id
        RETURN count(c) as document_count
    """
    params = {"node_id": node_id}
    count_result = graph.run(count_query, params).data()[0]
    document_count = count_result['document_count']

    print(f"Number of documents: {document_count}")

    if document_count <= 10:
        content_query = """
            MATCH (n)-[:HAS_CONTENTS]->(c)
            WHERE id(n) = $node_id
            RETURN c.order, c.text
            ORDER BY c.order
            LIMIT 5
        """

    #else:
        #Vector search

    content_results = graph.run(content_query, params)
    contents = [record["c.text"] for record in content_results]

    return contents

def get_sibling_contents_dev(cur_pos):
    
    return

def generate_answer_dev(question, contents):
    # Prompt setting
    sys_prompt_template = "당신은 AWS에 정통한 전문 엔지니어입니다. 주어진 사전 정보만 활용하여, 사용자 질문에 답변을 생성하세요. 사전 정보로 주어지지 않은 내용에 대한 질문에는 모른다고 답변하세요."
    usr_prompt_template = "#사전 정보: {context}\n\n #사용자 질문:\n {question}"
    prompt = ChatPromptTemplate.from_messages([("system", sys_prompt_template), ("human",usr_prompt_template)])

    # Model setting
    model_kwargs = {
            "temperature": 0.5,
            "max_tokens": 4096
        }
    llm = ChatBedrock(model_id="anthropic.claude-3-5-sonnet-20240620-v1:0", region_name="us-west-2", model_kwargs=model_kwargs, streaming=True)   

    # Output setting
    parser = StrOutputParser()

    # Chain
    chain = prompt | llm | parser

    context = " ".join(contents)
    for chunk in chain.stream({"context": context, "question": question}):
        print(chunk, end="", flush=True)


question = "Bedrock Agent 성능 최적화 방법"
target_node, next_step = select_subgraph_dev(question, graph)
print("1. target_node:", target_node)
cur_pos, target_node, next_step = traverse_child_dev(question, graph, target_node)
print("2. target_node:", target_node)
cur_pos, target_node, next_step = traverse_child_dev(question, graph, target_node)
print("3. target_node:", target_node)
cur_pos, target_node, next_step = traverse_child_dev(question, graph, target_node)
print("4. target_node:", target_node)
cur_pos, target_node, next_step = traverse_child_dev(question, graph, target_node)
print("5. target_node:", target_node)
cur_pos, target_node, next_step = traverse_child_dev(question, graph, target_node)
print("6. target_node:", target_node)
contents = get_contents_dev(question, cur_pos)

generate_answer_dev(question, contents)



1. target_node: [0]
Node value: Amazon Bedrock
2. target_node: [1493, 508, 646]
Node value: Agents for Amazon Bedrock
3. target_node: [1718, 1509, 1497]
Node value: Customize an Amazon Bedrock agent
4. target_node: [1859, 1720, 1837]
Node value: Optimize performance for Amazon Bedrock agents
5. target_node: [1861]
Node value: Optimize performance for Amazon Bedrock agents using a single knowledge base
6. target_node: []
Number of documents: 7
Agents for Amazon Bedrock의 성능을 최적화하기 위해 다음과 같은 방법들을 사용할 수 있습니다:

1. 단일 지식 베이스 사용: 
   - 에이전트에 하나의 지식 베이스만 포함되어 있는지 확인합니다.

2. 액션 그룹 비활성화:
   - 모든 액션 그룹을 비활성화하거나 제거합니다.

3. 사용자 입력 요청 최소화:
   - 에이전트가 추가 정보를 사용자에게 요청하지 않도록 설정합니다.

4. 기본 오케스트레이션 프롬프트 템플릿 사용:
   - 오케스트레이션 프롬프트 템플릿을 기본값으로 설정합니다.

5. 콘솔에서 설정 확인:
   - User input 필드가 비활성화되어 있는지 확인
   - Knowledge bases 섹션에 하나의 지식 베이스만 있는지 확인
   - Action groups 섹션에 액션 그룹이 없는지 확인
   - Advanced prompts의 Orchestration 필드가 Default로 설정되어 있는지 확인

6. API를 통한 확인:
   - ListAgentKnowledgeBases 요청으로 단일 지식 베이스 확인
   - L